In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
model_name = 'artificial'
log_name = 'train'
with open('../transformed_event_logs/artificial_start_end_2_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)
known_resources = ['1', 'Clark', 'Jane', 'Joe', 'Karsten']
known_activities = ['DIAGNOSIS', 'QUALITY_CONTROL', 'REPAIR']

In [3]:
test_event_log

,concept:name,lifecycle:transition_start,time:timestamp_start,org:resource,case:concept:name,id_start,lifecycle:transition_complete,time:timestamp_complete,id_complete,duration,duration_seconds,seconds_in_day,DIAGNOSIS,QUALITY_CONTROL,REPAIR,1,Clark,Jane,Joe,Karsten
0,DIAGNOSIS,START,2020-01-01 03:57:40.044121+00:00,Jane,0,1,COMPLETE,2020-01-01 04:34:23.549454+00:00,2,0 days 00:36:43.505333,2203,14260,1,0,0,0,0,1,0,0
1,REPAIR,START,2020-01-01 04:34:23.549454+00:00,Joe,0,4,COMPLETE,2020-01-01 14:30:27.423999+00:00,5,0 days 09:56:03.874545,35763,16463,1,0,1,0,0,1,1,0
3,DIAGNOSIS,START,2020-01-01 08:16:35.844753+00:00,Jane,1,10,COMPLETE,2020-01-01 08:47:14.772217+00:00,11,0 days 00:30:38.927464,1838,29795,1,0,0,0,0,1,0,0
4,REPAIR,START,2020-01-01 08:47:14.772217+00:00,Karsten,1,13,COMPLETE,2020-01-01 13:27:22.316694+00:00,14,0 days 04:40:07.544477,16807,31634,1,0,1,0,0,1,0,1
5,QUALITY_CONTROL,START,2020-01-01 13:27:22.316694+00:00,Clark,1,16,COMPLETE,2020-01-01 17:21:42.825766+00:00,17,0 days 03:54:20.509072,14060,48442,1,1,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5400,DIAGNOSIS,START,2024-12-26 18:05:59.540931+00:00,Clark,1800,16201,COMPLETE,2024-12-26 19:13:26.892809+00:00,16202,0 days 01:07:27.351878,4047,65159,1,0,0,0,1,0,0,0
5399,QUALITY_CONTROL,START,2024-12-26 19:03:41.622915+00:00,Joe,1799,16198,COMPLETE,2024-12-26 21:39:17.312456+00:00,16199,0 days 02:35:35.689541,9335,68621,1,1,1,1,0,0,1,1
5401,REPAIR,START,2024-12-26 19:13:26.892809+00:00,Clark,1800,16204,COMPLETE,2024-12-27 08:23:46.946760+00:00,16205,0 days 13:10:20.053951,47420,69206,1,0,1,0,2,0,0,0
5402,QUALITY_CONTROL,START,2024-12-27 08:23:46.946760+00:00,Jane,1800,16207,COMPLETE,2024-12-27 11:10:44.009123+00:00,16208,0 days 02:46:57.062363,10017,30226,1,1,1,0,2,1,0,0


In [4]:
N = 10000
n_processes = 10
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [5]:
drbart_model_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_Normal_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A = evaluator_A.sample_cases(False, True)

  0%|                                                            | 0/1802 [00:00<?, ?it/s]

  0%|                                                  | 1/1802 [00:10<5:23:14, 10.77s/it]

  0%|▏                                                 | 7/1802 [00:18<1:08:35,  2.29s/it]

  0%|▎                                                   | 9/1802 [00:18<49:25,  1.65s/it]

  1%|▎                                                  | 11/1802 [00:19<39:21,  1.32s/it]

...

 99%|████████████████████████████████████████████████▌| 1788/1802 [01:58<00:00, 14.06it/s]

 99%|████████████████████████████████████████████████▋| 1790/1802 [01:58<00:01, 11.52it/s]

100%|████████████████████████████████████████████████▊| 1797/1802 [01:59<00:00, 16.65it/s]

100%|████████████████████████████████████████████████▉| 1800/1802 [01:59<00:00, 17.93it/s]

100%|█████████████████████████████████████████████████| 1802/1802 [01:59<00:00, 15.12it/s]

In [6]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

Decimal('0.01560398280535559698638707594')

In [7]:
np.mean(get_pscores(likelihoods_A))

np.float64(11030.634213305606)

In [8]:
drbart_model_A_R = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource/',
                     strict_parser=True)
evaluator_A_R = conduct_evaluation.ConductEvaluation(drbart_model_A_R, SampleOutcomes_DRBART_Normal_A_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R = evaluator_A_R.sample_cases(False, True)

  0%|                                                            | 0/1802 [00:00<?, ?it/s]

  0%|                                                  | 1/1802 [00:16<8:17:11, 16.56s/it]

  0%|                                                  | 2/1802 [00:17<3:43:42,  7.46s/it]

  1%|▎                                                  | 11/1802 [00:18<28:51,  1.03it/s]

  1%|▎                                                  | 12/1802 [00:20<32:56,  1.10s/it]

...

 99%|████████████████████████████████████████████████▋| 1791/1802 [02:02<00:00, 13.66it/s]

100%|████████████████████████████████████████████████▊| 1794/1802 [02:03<00:00, 12.94it/s]

100%|████████████████████████████████████████████████▊| 1796/1802 [02:03<00:00, 13.82it/s]

100%|████████████████████████████████████████████████▉| 1801/1802 [02:03<00:00, 18.84it/s]

100%|█████████████████████████████████████████████████| 1802/1802 [02:03<00:00, 14.60it/s]

In [9]:
np.mean([v.ln() for v in likelihoods_A_R[0].values()])

Decimal('0.2484964933485186374001278465')

In [10]:
np.mean(get_pscores(likelihoods_A_R))

np.float64(9806.944375306299)

In [11]:
drbart_model_R_S = DRBART(parser_dir = '../../../models/'+model_name+'/resource_seconds-in-day/',
                     strict_parser=True)
evaluator_R_S = conduct_evaluation.ConductEvaluation(drbart_model_R_S, SampleOutcomes_DRBART_Normal_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_S = evaluator_R_S.sample_cases(False, True)

  0%|                                                            | 0/1802 [00:00<?, ?it/s]

  0%|                                                  | 1/1802 [00:11<5:41:36, 11.38s/it]

  0%|                                                  | 2/1802 [00:19<4:45:14,  9.51s/it]

  1%|▎                                                  | 11/1802 [00:21<38:29,  1.29s/it]

  1%|▎                                                  | 13/1802 [00:21<31:11,  1.05s/it]

...

 99%|████████████████████████████████████████████████▋| 1792/1802 [01:52<00:00, 19.26it/s]

100%|████████████████████████████████████████████████▊| 1795/1802 [01:52<00:00, 19.90it/s]

100%|████████████████████████████████████████████████▉| 1798/1802 [01:52<00:00, 16.16it/s]

100%|█████████████████████████████████████████████████| 1802/1802 [01:52<00:00, 19.51it/s]

100%|█████████████████████████████████████████████████| 1802/1802 [01:52<00:00, 15.99it/s]

In [12]:
np.mean([v.ln() for v in likelihoods_R_S[0].values()])

Decimal('0.1339985077135421053944694550')

In [13]:
np.mean(get_pscores(likelihoods_R_S))

np.float64(11562.278712344216)

In [14]:
drbart_model_R = DRBART(parser_dir = '../../../models/'+model_name+'/resource/',
                     strict_parser=True)
evaluator_R = conduct_evaluation.ConductEvaluation(drbart_model_R, SampleOutcomes_DRBART_Normal_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R = evaluator_R.sample_cases(False, True)

  0%|                                                            | 0/1802 [00:00<?, ?it/s]

  0%|                                                  | 1/1802 [00:11<5:51:11, 11.70s/it]

  0%|                                                  | 4/1802 [00:11<1:07:27,  2.25s/it]

  1%|▎                                                  | 11/1802 [00:21<49:08,  1.65s/it]

  1%|▎                                                  | 12/1802 [00:21<43:09,  1.45s/it]

...

 99%|████████████████████████████████████████████████▋| 1791/1802 [01:38<00:00, 17.25it/s]

100%|████████████████████████████████████████████████▊| 1796/1802 [01:38<00:00, 20.76it/s]

100%|████████████████████████████████████████████████▉| 1799/1802 [01:38<00:00, 17.06it/s]

100%|████████████████████████████████████████████████▉| 1801/1802 [01:38<00:00, 17.45it/s]

100%|█████████████████████████████████████████████████| 1802/1802 [01:38<00:00, 18.25it/s]

In [15]:
np.mean([v.ln() for v in likelihoods_R[0].values()])

Decimal('0.03739018906216750276758025934')

In [16]:
np.mean(get_pscores(likelihoods_R))

np.float64(13287.661498856269)

In [17]:
drbart_model_A_R_S = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day/',
                     strict_parser=True)
evaluator_A_R_S = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S, SampleOutcomes_DRBART_Normal_A_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S = evaluator_A_R_S.sample_cases(False, True)

  0%|                                                            | 0/1802 [00:00<?, ?it/s]

  0%|                                                  | 1/1802 [00:07<3:54:45,  7.82s/it]

  0%|                                                  | 3/1802 [00:07<1:02:15,  2.08s/it]

  0%|▏                                                   | 5/1802 [00:08<31:20,  1.05s/it]

  1%|▎                                                  | 11/1802 [00:15<35:37,  1.19s/it]

...

 99%|████████████████████████████████████████████████▌| 1785/1802 [01:37<00:01, 15.97it/s]

 99%|████████████████████████████████████████████████▋| 1790/1802 [01:37<00:00, 19.64it/s]

100%|████████████████████████████████████████████████▊| 1795/1802 [01:38<00:00, 16.52it/s]

100%|████████████████████████████████████████████████▉| 1800/1802 [01:38<00:00, 19.85it/s]

100%|█████████████████████████████████████████████████| 1802/1802 [01:38<00:00, 18.31it/s]

In [18]:
np.mean([v.ln() for v in likelihoods_A_R_S[0].values()])

Decimal('0.2571375123984412439695113196')

In [19]:
np.mean(get_pscores(likelihoods_A_R_S))

np.float64(10193.746077680995)

In [20]:
drbart_model_A_R_S_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_A_R_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_AC, SampleOutcomes_DRBART_Normal_A_R_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_AC = evaluator_A_R_S_AC.sample_cases(False, True)

  0%|                                                            | 0/1802 [00:00<?, ?it/s]

  0%|                                                  | 1/1802 [00:08<4:25:54,  8.86s/it]

  0%|                                                  | 3/1802 [00:08<1:10:23,  2.35s/it]

  1%|▎                                                  | 11/1802 [00:17<39:36,  1.33s/it]

  1%|▎                                                  | 13/1802 [00:17<31:15,  1.05s/it]

...

 99%|████████████████████████████████████████████████▋| 1791/1802 [01:38<00:00, 16.52it/s]

100%|████████████████████████████████████████████████▊| 1794/1802 [01:38<00:00, 17.11it/s]

100%|████████████████████████████████████████████████▉| 1799/1802 [01:38<00:00, 21.87it/s]

100%|█████████████████████████████████████████████████| 1802/1802 [01:39<00:00, 16.53it/s]

100%|█████████████████████████████████████████████████| 1802/1802 [01:39<00:00, 18.18it/s]

In [21]:
np.mean([v.ln() for v in likelihoods_A_R_S_AC[0].values()])

Decimal('0.2787448436170949986174899664')

In [22]:
np.mean(get_pscores(likelihoods_A_R_S_AC))

np.float64(9941.593139598283)

In [23]:
drbart_model_A_R_S_RC_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC_AC, SampleOutcomes_DRBART_Normal_A_R_S_RC_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_RC_AC = evaluator_A_R_S_RC_AC.sample_cases(False, True)

  0%|                                                            | 0/1802 [00:00<?, ?it/s]

  0%|                                                  | 1/1802 [00:09<4:40:44,  9.35s/it]

  0%|                                                  | 3/1802 [00:09<1:14:00,  2.47s/it]

  0%|▏                                                   | 8/1802 [00:09<20:51,  1.43it/s]

  1%|▎                                                  | 11/1802 [00:21<56:59,  1.91s/it]

...

100%|████████████████████████████████████████████████▊| 1793/1802 [01:38<00:00, 19.33it/s]

100%|████████████████████████████████████████████████▊| 1796/1802 [01:38<00:00, 18.53it/s]

100%|████████████████████████████████████████████████▉| 1798/1802 [01:38<00:00, 16.19it/s]

100%|█████████████████████████████████████████████████| 1802/1802 [01:38<00:00, 19.74it/s]

100%|█████████████████████████████████████████████████| 1802/1802 [01:38<00:00, 18.22it/s]

In [24]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC_AC[0].values()])

Decimal('0.2748413552396342672834682986')

In [25]:
np.mean(get_pscores(likelihoods_A_R_S_RC_AC))

np.float64(8396.945068078916)

In [26]:
drbart_model_A_R_S_RC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC, SampleOutcomes_DRBART_Normal_A_R_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_RC = evaluator_A_R_S_RC.sample_cases(False, True)

  0%|                                                            | 0/1802 [00:00<?, ?it/s]

  0%|                                                  | 1/1802 [00:08<4:21:57,  8.73s/it]

  0%|                                                  | 2/1802 [00:08<1:50:04,  3.67s/it]

  0%|▎                                                   | 9/1802 [00:08<16:26,  1.82it/s]

  1%|▎                                                  | 12/1802 [00:17<39:55,  1.34s/it]

...

 99%|████████████████████████████████████████████████▋| 1791/1802 [01:38<00:00, 17.19it/s]

100%|████████████████████████████████████████████████▊| 1796/1802 [01:39<00:00, 18.90it/s]

100%|████████████████████████████████████████████████▉| 1798/1802 [01:39<00:00, 17.64it/s]

100%|████████████████████████████████████████████████▉| 1801/1802 [01:39<00:00, 17.12it/s]

100%|█████████████████████████████████████████████████| 1802/1802 [01:39<00:00, 18.13it/s]

In [27]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC[0].values()])

Decimal('0.3898494322177546494885940142')

In [28]:
np.mean(get_pscores(likelihoods_A_R_S_RC))

np.float64(8066.157753618569)

In [29]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_R' : likelihoods_A_R,
    'drbart_model_R' : likelihoods_R,
    'drbart_model_R_S' : likelihoods_R_S,
    'drbart_model_A_R_S' : likelihoods_A_R_S,
    'drbart_model_A_R_S_AC' : likelihoods_A_R_S_AC,
    'drbart_model_A_R_S_RC' : likelihoods_A_R_S_RC,
    'drbart_model_A_R_S_RC_AC' : likelihoods_A_R_S_RC_AC
}

with open('./'+model_name+'_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)